# Практическое задание к уроку 4. Тематическое моделирование. EM-алгоритм

Задача тематическое моделирование. Продолжаем исследование датасета с твитами.

Скачиваем датасет (источник): положительные, отрицательные.

или можно через ноутбук

!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv 
!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv

как альтернатива можно скачать данные из Роспотребнадзора https://zpp.rospotrebnadzor.ru/Forum/Appeals

для этого берём ноутбук parse_rospotrebnadzor.ipynb устанавливаем количество скачанных страниц больше не 50-сят хотябы 500 и для анализа берём только вопросы так как ответы есть не всегда

Что надо сделать:

1. объединить в одну выборку (это только для твитов), для роспотребнадзора сформировать датасет из вопросов

2. провести исследование и выявить тематики о которых говорят в твитах (для твитов), а для роспотребнадзора так же выявить тематики о которых люди пишут проанализировать

3. сделать визуализацию кластеров тематик

4. проинтерпритировать получившиеся тематики

In [1]:
!pip install pyLDAvis

In [2]:
# Импорт библиотек

import pandas as pd
import re
import numpy as np
from nltk.corpus import stopwords
from pymystem3 import Mystem
import nltk
nltk.download('stopwords')
import pymorphy2
from gensim.models import *
from gensim import corpora
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import matplotlib.pyplot as plt
import warnings
def fxn():
    warnings.warn("deprecated", DeprecationWarning)
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/alenakukhta/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## 1. Объединить в одну выборку (это только для твитов), для роспотребнадзора сформировать датасет из вопросов

In [3]:
# Загрузка данных

positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['text'])
positive['label'] = ['positive'] * len(positive)

negative = pd.read_csv('negative.csv', sep=';', usecols=[3], names=['text'])
negative['label'] = ['negative'] * len(negative)

# Объединение двух датафреймов

df = pd.concat([positive, negative], ignore_index=True)

df.head()

,text,label
0,"@first_timee хоть я и школота, но поверь, у на...",positive
1,"Да, все-таки он немного похож на него. Но мой ...",positive
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,positive
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",positive
4,@irina_dyshkant Вот что значит страшилка :D\nН...,positive


In [4]:
# Информация о датасете
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226834 entries, 0 to 226833
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    226834 non-null  object
 1   label   226834 non-null  object
dtypes: object(2)
memory usage: 3.5+ MB


## 2. Провести исследование и выявить тематики о которых говорят в твитах (для твитов), а для роспотребнадзора так же выявить тематики о которых люди пишут проанализировать

In [5]:
### Препроцессинг текста

# Компеляция объекта (1 или более букв)
words_regex = re.compile('\w+') 

# Формирование списка токенов из букв и длины >3
def find_words(text, regex = words_regex):
    tokens = regex.findall(text) # поиск всех слов в тексте --> получение списка токенов
    return [w for w in tokens if w.isalpha() and len(w)>= 4] # выбор токетов, состоящих из букв длиною > 3

morph = pymorphy2.MorphAnalyzer() # морфологический анализ
stopwords_list = stopwords.words('russian') # список стоп-слов

# Формирование списка нормальных форм токенов: удаление стопслов и токенов из небукв
def lemmatize(words, lemmer = morph, stopwords = stopwords_list):
    lemmas = [lemmer.parse(w)[0].normal_form for w in words] #список нормальных форм для всех токенов
    return [w for w in lemmas if not w in stopwords and w.isalpha()] #удаляем из списка нормальных форм стоп-слова и токены с небуквой(ми)

# Очищение токены от нецензурной лексики и ненужных слов
def clear_tokens(tokens):
    string_tokens = ' '.join(tokens)
    string_tokens = re.sub(r'[A-za-z]', ' ',  string_tokens) 
    string_tokens = re.sub(r'\w+ще\b', ' ',  string_tokens) 
    string_tokens = re.sub(r'\bбл\w+', ' ',  string_tokens) 
    string_tokens = re.sub(r'значит', ' ',  string_tokens) 
    string_tokens = re.sub(r'почему', ' ',  string_tokens) 
    string_tokens = re.sub(r'свой', ' ',  string_tokens) 
    string_tokens = re.sub(r'капец', ' ',  string_tokens) 
    string_tokens = re.sub(r'который', ' ',  string_tokens)
    string_tokens = re.sub(r'это', ' ',  string_tokens)
    string_tokens = re.sub(r'весь', ' ',  string_tokens)
    string_tokens = re.sub(r'\bидио\w+', ' ',  string_tokens) 
    string_tokens = re.sub(r'\bпид\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\bхуй\w+', ' ', string_tokens)
    string_tokens = re.sub(r'\w+xу.\b', ' ',  string_tokens)
    string_tokens = re.sub(r'\w+xу\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\w+но\b', ' ',  string_tokens)
    string_tokens = re.sub(r'\bпоп\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\bпиз\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\bгов\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\bфиг\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'сука', ' ',  string_tokens)
    string_tokens = re.sub(r'очень', ' ',  string_tokens)
    string_tokens = re.sub(r'лох', ' ',  string_tokens)
    string_tokens = re.sub(r'\bпи.де\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'нету', ' ',  string_tokens)
    string_tokens = re.sub(r'ой', ' ',  string_tokens)
    string_tokens = re.sub(r'\bхот\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\bпрост\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\bника\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\w+ть\b', ' ',  string_tokens)
    string_tokens = re.sub(r'мочь', ' ',  string_tokens)
    clear_tokens = string_tokens.split()
    clear_tokens = [w for w in clear_tokens if len(w)>2]
    return clear_tokens

def preprocess(text):
    return (clear_tokens(lemmatize(find_words(text))))

<>:4: DeprecationWarning: invalid escape sequence \w
<>:4: DeprecationWarning: invalid escape sequence \w
/var/folders/64/g6jhnpm16ql1vgrrbrbd_yd00000gn/T/ipykernel_14132/10130644.py:4: DeprecationWarning: invalid escape sequence \w
  words_regex = re.compile('\w+')
/Users/alenakukhta/opt/anaconda3/lib/python3.9/site-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
/Users/alenakukhta/opt/anaconda3/lib/python3.9/site-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
/Users/alenakukhta/opt/anaconda3/lib/python3.9/site-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.sig

In [6]:
# Предобработка твитов
df['text'] = df['text'].apply(preprocess)
df.head()

,text,label
0,"[школотый, поверь, самый, общество, предмет, тип]",positive
1,"[таки, немного, похожий, мальчик, хороший]",positive
2,[испугаться],positive
3,"[угол, голод, порция]",positive
4,"[страшилка, создаться, ощущение, автор]",positive


#### Модель Latent Dirichlet Allocation (LDA)

In [7]:
# Формирование словаря уникальных токенов датасета

dictionary = corpora.Dictionary(df['text']) # словарь токенов и их индексов (на вход список списков)

dictionary.filter_extremes(no_below = 10, no_above = 0.9, keep_n=None) # исключение слов, встречающихся 
                                                                    # реже 10 раз или составляют более 0.9 словаря 

dictionary.save('tweet.dict')

len(dictionary.keys()) # длина словаря из токенов

8172

In [8]:
# Векторизация твитов

corpus = [dictionary.doc2bow(text) for text in df['text']] # для каждого списка токенов возвращается кортеж 
                                                            # (индекс и количество повторений токена в твите)

corpora.MmCorpus.serialize('tweet.model', corpus) # итерация по корпусу (записывается смещение байта каждого твита)
                                                    # полученная структура индексов сохраняется в файл

In [9]:
# Итерация по словарю

mm = corpora.MmCorpus('tweet.model') # корпус имеет произвольный доступ

print(df['text'].values[0])
print(mm[0]) #  извлечение документа с номером 0

['школотый', 'поверь', 'самый', 'общество', 'предмет', 'тип']
[(0, 1.0), (1, 1.0), (2, 1.0), (3, 1.0), (4, 1.0), (5, 1.0)]


In [10]:
# Обучение модели

lda = ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=3)

In [11]:
# Определение вероятных тем

lda.show_topics(num_topics=3, num_words=20, formatted=True)

[(0,
  '0.021*"завтра" + 0.015*"дом" + 0.013*"никто" + 0.012*"мама" + 0.012*"жизнь" + 0.010*"работа" + 0.009*"прийти" + 0.009*"каждый" + 0.009*"скоро" + 0.007*"дело" + 0.007*"прийтись" + 0.007*"бояться" + 0.007*"закончиться" + 0.006*"боль" + 0.006*"правда" + 0.006*"твиттер" + 0.006*"ужасный" + 0.006*"фотка" + 0.006*"нужный" + 0.005*"город"'),
 (1,
  '0.024*"день" + 0.023*"сегодня" + 0.018*"больш" + 0.018*"школа" + 0.017*"время" + 0.013*"утро" + 0.013*"жаль" + 0.012*"друг" + 0.011*"последний" + 0.010*"урок" + 0.010*"ночь" + 0.009*"пока" + 0.008*"дом" + 0.008*"остаться" + 0.008*"найти" + 0.008*"человек" + 0.008*"втор" + 0.007*"жалко" + 0.007*"нравиться" + 0.007*"неделя"'),
 (2,
  '0.020*"хороший" + 0.017*"новый" + 0.013*"человек" + 0.012*"день" + 0.012*"час" + 0.012*"настроение" + 0.011*"год" + 0.011*"сегодня" + 0.010*"любимый" + 0.010*"голова" + 0.010*"самый" + 0.009*"экзамен" + 0.008*"завтра" + 0.008*"каникулы" + 0.008*"фильм" + 0.008*"вчера" + 0.007*"вроде" + 0.007*"рука" + 0.006*"сне

In [12]:
# Оценка качества полученной модели

print(lda.log_perplexity(corpus))

-8.0153255101143


In [13]:
print('Персплексия: ', np.exp(lda.log_perplexity(corpus)))

Персплексия:  0.0003303605367699587


## 3. Сделать визуализацию кластеров тематик

In [14]:
vis_data = gensimvis.prepare(lda, corpus, dictionary)
pyLDAvis.display(vis_data)

/Users/alenakukhta/opt/anaconda3/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(
/Users/alenakukhta/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/alenakukhta/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/alenakukhta/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/alenakukhta

## 4. Проинтерпритировать получившиеся тематики

Примерные названия тем твитов:

Тема 1. Учеба и впечатления школьника.

Тема 2. Личная жизнь студентки.

Тема 3. Эмоции работника.

3 темы наиболее актуальное количество, поскольку при большем значении возможны их наслоения.